In [1]:
import pandas as pd 
from numpy import int16

In [2]:
#DODAC ROZDIZAŁ NA ZLECENI Z PŁYTĄ PROSTOKĄTNĄ 1250x2500MM

In [3]:
#@title Funkcje
def generuj_tableke_na_cnc(_zlecenie):
  # zlecenie_ngl = _zlecenie.split("\n")[1].split("\t")[1:]
  zlecenie_ngl = ["MATERIAŁ",	"NUMER",	"ILOŚĆ",	"WYMIAR",	"OZNACZ.",	"CZAS"]
  zlecenie = list()
  for r in _zlecenie.split("\n")[2:]:
    r = r.split("\t")
    if len(r) == 7 and r[0] not in ["", "L.P."]:
      zlecenie.append(r[1:])

  df = pd.DataFrame(zlecenie, columns=zlecenie_ngl)

  df["GRUBOSC"] = df["MATERIAŁ"].apply(lambda x: x[-4:])


  def mat_typ(x):
    if "Sklejka" in x:
      return "SKL"
    elif "wiórowa" in x:
      return "PWI"

  def szer(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c not in "* ":
        new_str += c
    return new_str.strip().split("x")[0]

  def dlug(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c != "*" or c != " ":
        new_str += c
    return new_str.strip().split("x")[1]

  df["SZER"] = df["WYMIAR"].apply(szer).astype(int16)
  df["DLUG"] = df["WYMIAR"].apply(dlug).astype(int16)
  df["MAT_TYP"] = df["MATERIAŁ"].apply(mat_typ)
  df["ILOSC"] = df["ILOŚĆ"].astype(int16)

  return df

def drukuj_dane_cnc(df,MAT):

  _naglowek = "index$,material$,krok,sloje,grubosc,ilosc,mosty,nazwafmc$,kolor,fmcinclude$\n"
  content = list()
  content.append(_naglowek)
  # print(pwi_naglowek)
  for i in df[["NUMER", "ILOSC"]].values.tolist():
    if MAT == "S":
      a = [f"{i[0]}_SKL_X1_Msklejka,Sklejka,90,0,0,{i[1]},0,SKL",f"{i[0]}_SKL_X1_MSklejka.fmc,2,SKL","kontury"]
    elif MAT == "P":
      a = [f"{i[0]}_PWI_X1_MWIOR,WIOR,90,0,0,{i[1]},0,WIOR",f"{i[0]}_PWI_X1_MWIOR.fmc,2,WIOR","kontury"]
    b = chr(92).join(a)+chr(92)+"\n"
    # b = "/".join(a)+"/"+"\n"
    # print(b)
    content.append(b)

  content[-1] = content[-1].replace("\n", "")
  return content

def podziel_zlecenie(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df.groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:
      if f[0] == "PWI":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1])], "P")

      elif f[0] == "SKL":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1]) & (df.DLUG <= 1525)], "S")

      fi = open(f"{n}_{f[0]}_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

def podziel_zlecenie_SKLP(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df[(df.MAT_TYP=="SKL")].groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:

      content = drukuj_dane_cnc(df[(df.MAT_TYP=="SKL") & (df.GRUBOSC == f[1]) & (df.DLUG > 1525)], "S")

      fi = open(f"{n}_SKLP_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

In [4]:

#wzór na zlecenie
# """
# L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
# 1	Płyta wiórowa 16mm	1278	40	*** 115x245	CN	20
# 2	Płyta wiórowa 16mm	1279	80	*** 110x222	CN	40
# 3	Płyta wiórowa 16mm	1393	40	*** 158x260	CN	20
_zlecenie = {

"HOR_5_1_24":
              """
L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Sklejka 18mm	1545	165	*** 120x960	CN	46:45 (17)
2	Sklejka 18mm	1549	100	*** 120x1110	CN	23:20 (14)
3	Sklejka 18mm	1551	40	*** 120x1110	CN	9:20 (14)
4	Sklejka 18mm	1933	0	*** 100x161	CN	25:20 (19)
5	Sklejka 18mm	1934	0	*** 100x155	CN	80:40 ()
6	Sklejka 18mm	1936	190	*** 120x960	CN	60:10 (19)





 """

}


In [5]:
podziel_zlecenie(_zlecenie)

In [23]:
podziel_zlecenie_SKLP(_zlecenie)